In [1]:
from pydrake.all import (Meshcat, Simulator)
from manipulation.station import LoadScenario, MakeHardwareStation

from pydrake.planning import KinematicTrajectoryOptimization
from pydrake.solvers import Solve, SolverOptions, SnoptSolver
import numpy as np
import time

meshcat = Meshcat()



INFO:drake:Meshcat listening for connections at http://localhost:7000


In [2]:
scenario_data = """
directives:
- add_model:
    name: mobile_iiwa
    file: package://manipulation/mobile_iiwa14_primitive_collision.urdf
model_drivers:
    mobile_iiwa: !InverseDynamicsDriver {}
"""

scenario = LoadScenario(data=scenario_data)
station = MakeHardwareStation(scenario, meshcat)
sim = Simulator(station)
context = sim.get_mutable_context()

x0 = station.GetOutputPort("mobile_iiwa.state_estimated").Eval(context)
station.GetInputPort("mobile_iiwa.desired_state").FixValue(context, x0)
sim.AdvanceTo(0.1)

In [21]:
# Create a room environment in meshcat
from pydrake.geometry import Box, Cylinder, Rgba
from pydrake.all import RigidTransform
import numpy as np

# Floor (large flat box)
floor_geometry = Box(16, 16, 0.4)  # width, depth, height (4x larger)
floor_pose = RigidTransform([0, 0, -0.2])  # slightly below origin
meshcat.SetObject("floor", floor_geometry, rgba=Rgba(0.5, 0.5, 0.5, 0.8))
meshcat.SetTransform("floor", floor_pose)

# Walls (4 walls)
wall_height = 10.0
wall_thickness = 0.4

# Front wall (x-direction)
front_wall = Box(16, wall_thickness, wall_height)
front_pose = RigidTransform([0, 8, wall_height/2])
meshcat.SetObject("front_wall", front_wall, rgba=Rgba(0.8, 0.8, 0.8, 0.6))
meshcat.SetTransform("front_wall", front_pose)

# Back wall
back_wall = Box(16, wall_thickness, wall_height)
back_pose = RigidTransform([0, -8, wall_height/2])
meshcat.SetObject("back_wall", back_wall, rgba=Rgba(0.8, 0.8, 0.8, 0.6))
meshcat.SetTransform("back_wall", back_pose)

# Left wall (y-direction)
left_wall = Box(wall_thickness, 16, wall_height)
left_pose = RigidTransform([-8, 0, wall_height/2])
meshcat.SetObject("left_wall", left_wall, rgba=Rgba(0.8, 0.8, 0.8, 0.6))
meshcat.SetTransform("left_wall", left_pose)

# Right wall
right_wall = Box(wall_thickness, 16, wall_height)
right_pose = RigidTransform([8, 0, wall_height/2])
meshcat.SetObject("right_wall", right_wall, rgba=Rgba(0.8, 0.8, 0.8, 0.6))
meshcat.SetTransform("right_wall", right_pose)

print("Room created in meshcat!")

Room created in meshcat!


In [13]:
# Add 3 fixed tables to the room
table_height = 1.0
table_length = 2.0
table_width = 1.5
leg_thickness = 0.1

# Fixed table positions (away from center to avoid collision with KUKA iiwa)
table_positions = [
    (-4, -4),  # Table 1 - back left corner
    (3, -4),   # Table 2 - back right corner
    (1, 3)     # Table 3 - front right corner
]

for i, (x, y) in enumerate(table_positions):
    # Create table top
    table_top = Box(table_length, table_width, 0.05)
    table_pose = RigidTransform([x, y, table_height])
    meshcat.SetObject(f"table_{i}/top", table_top, rgba=Rgba(0.6, 0.3, 0.1, 0.9))
    meshcat.SetTransform(f"table_{i}/top", table_pose)
    
    # Create 4 legs
    leg = Box(leg_thickness, leg_thickness, table_height)
    leg_offset = 0.3
    
    for j, (dx, dy) in enumerate([(-leg_offset, -leg_offset), (-leg_offset, leg_offset), 
                                   (leg_offset, -leg_offset), (leg_offset, leg_offset)]):
        leg_pose = RigidTransform([x + dx, y + dy, table_height/2])
        meshcat.SetObject(f"table_{i}/leg_{j}", leg, rgba=Rgba(0.4, 0.2, 0.0, 0.9))
        meshcat.SetTransform(f"table_{i}/leg_{j}", leg_pose)

print("3 fixed tables added to the room (away from center)!")

3 fixed tables added to the room (away from center)!


In [22]:
# Add collision geometry for tables
from pydrake.geometry import GeometryInstance
from pydrake.common import RandomGenerator

# Register collision geometries for each table
for i, (x, y) in enumerate(table_positions):
    # Single box for table top as collision geometry
    collision_box = Box(table_length, table_width, 0.05)
    pose = RigidTransform([x, y, table_height])
    
    # You would add this to the plant's collision detection
    # For now, this marks the collision geometry visually
    meshcat.SetObject(f"table_{i}/collision", collision_box, rgba=Rgba(1.0, 0.0, 0.0, 0.1))
    meshcat.SetTransform(f"table_{i}/collision", pose)

print("Collision geometry added for tables!")

Collision geometry added for tables!


In [35]:
# Define collision-free waypoints with tighter spacing for full room exploration
waypoints = []
spacing = 2.0  # 2m spacing instead of 3m for denser coverage
for x in np.arange(-6, 7, spacing):
    for y in np.arange(-6, 7, spacing):
        waypoints.append([x, y])

# Filter waypoints - increase safety margin to 2.5m to keep robot well away
margin = 2.5
safe_waypoints = []
for wp in waypoints:
    safe = True
    for tx, ty in table_positions:
        dist = np.sqrt((wp[0]-tx)**2 + (wp[1]-ty)**2)
        if dist < margin:
            safe = False
            break
    if safe:
        safe_waypoints.append(wp)

waypoints = safe_waypoints
print(f"Waypoints: {len(waypoints)} safe locations")

# Visualize in meshcat
for i, wp in enumerate(waypoints):
    meshcat.SetObject(f"wp_{i}", Box(0.15, 0.15, 0.15), rgba=Rgba(0, 1, 0, 0.7))
    meshcat.SetTransform(f"wp_{i}", RigidTransform([wp[0], wp[1], 0.2]))

Waypoints: 34 safe locations


In [38]:
# Execute trajectory planning and animation through waypoints
plant = station.GetSubsystemByName("plant")
plant_context = plant.GetMyMutableContextFromRoot(context)

# Get base joint indices for mobile iiwa
ix = plant.GetJointByName("iiwa_base_x").position_start()
iy = plant.GetJointByName("iiwa_base_y").position_start()
iz = plant.GetJointByName("iiwa_base_z").position_start()

# Move robot to origin first
q_current = plant.GetPositions(plant_context).copy()
q_current[ix] = 0.0
q_current[iy] = 0.0
plant.SetPositions(plant_context, q_current)
station.ForcedPublish(context)
print("Robot moved to origin (0, 0)")

q0 = plant.GetPositions(plant_context).copy()
qlo = plant.GetPositionLowerLimits()
qhi = plant.GetPositionUpperLimits()
vlo = plant.GetVelocityLowerLimits()
vhi = plant.GetVelocityUpperLimits()
nq = plant.num_positions()

# Set velocity limits for base motion
vlo[[ix, iy, iz]] = -1.0
vhi[[ix, iy, iz]] = 1.0

# Plan and execute trajectories between consecutive waypoints
all_trajectories = []

for wp_idx in range(len(waypoints)):
    q_start = plant.GetPositions(plant_context).copy()
    
    # Set goal position at next waypoint
    q_goal = q_start.copy()
    q_goal[ix] = waypoints[wp_idx][0]
    q_goal[iy] = waypoints[wp_idx][1]
    
    # Trajectory optimization setup
    N = 20
    min_t = 1.0
    max_t = 10.0
    eps = 0.01  # Increased tolerance from 1e-4 to 0.01 for flexibility
    
    kto = KinematicTrajectoryOptimization(nq, N)
    prog = kto.get_mutable_prog()
    
    kto.AddDurationCost(1.0)
    kto.AddDurationConstraint(min_t, max_t)
    kto.AddPositionBounds(qlo, qhi)
    kto.AddVelocityBounds(vlo, vhi)
    kto.AddPathPositionConstraint(q_start - eps, q_start + eps, 0)
    kto.AddPathPositionConstraint(q_goal - eps, q_goal + eps, 1)
    
    # Solve without collision constraints (waypoints already filtered for safety)
    solver = SnoptSolver()
    opts = SolverOptions()
    opts.SetOption(solver.id(), "minor feasibility tolerance", 1e-4)
    res = solver.Solve(prog, solver_options=opts)
    
    if not res.is_success():
        print(f"Trajectory optimization failed for waypoint {wp_idx} - skipping")
        continue  # Skip this waypoint and continue to next one
    
    traj = kto.ReconstructTrajectory(res)
    all_trajectories.append(traj)
    
    # Execute trajectory
    T = traj.end_time()
    Ts = np.linspace(0.0, T, int(T * 30))  # 30 Hz animation
    
    for t in Ts:
        q = traj.value(t).ravel()
        plant.SetPositions(plant_context, q)
        station.ForcedPublish(context)
        time.sleep(0.01)
    
    # Finalize position at waypoint
    plant.SetPositions(plant_context, traj.value(T).ravel())
    station.ForcedPublish(context)
    print(f"Reached waypoint {wp_idx}: ({waypoints[wp_idx][0]}, {waypoints[wp_idx][1]})")

print(f"Exploration complete! Visited {len(all_trajectories)} waypoints.")

Robot moved to origin (0, 0)
Reached waypoint 0: (-6.0, -6.0)
Reached waypoint 0: (-6.0, -6.0)
Reached waypoint 1: (-6.0, -2.0)
Reached waypoint 1: (-6.0, -2.0)
Reached waypoint 2: (-6.0, 0.0)
Reached waypoint 2: (-6.0, 0.0)
Reached waypoint 3: (-6.0, 2.0)
Reached waypoint 3: (-6.0, 2.0)
Reached waypoint 4: (-6.0, 4.0)
Reached waypoint 4: (-6.0, 4.0)
Reached waypoint 5: (-6.0, 6.0)
Reached waypoint 5: (-6.0, 6.0)
Reached waypoint 6: (-4.0, 0.0)
Reached waypoint 6: (-4.0, 0.0)
Reached waypoint 7: (-4.0, 2.0)
Reached waypoint 7: (-4.0, 2.0)
Reached waypoint 8: (-4.0, 4.0)
Reached waypoint 8: (-4.0, 4.0)
Reached waypoint 9: (-4.0, 6.0)
Trajectory optimization failed for waypoint 10 - skipping
Reached waypoint 9: (-4.0, 6.0)
Trajectory optimization failed for waypoint 10 - skipping
Reached waypoint 11: (-2.0, -2.0)
Reached waypoint 11: (-2.0, -2.0)
Reached waypoint 12: (-2.0, 0.0)
Reached waypoint 12: (-2.0, 0.0)
Reached waypoint 13: (-2.0, 2.0)
Reached waypoint 13: (-2.0, 2.0)
Reached way